In [1]:
import sys

sys.path.append("..")
from omegaconf import OmegaConf
from sgm.util import instantiate_from_config

config = OmegaConf.load(
    "/vol/paramonos2/projects/antoni/code/Personal/generative-models/configs/inference/sd_1_5.yaml"
)
print(config)
config["model"]["params"][
    "ckpt_path"
] = "/vol/paramonos2/projects/antoni/code/Personal/generative-models/checkpoints/leosamsFilmgirlUltra_ultraBaseModel.safetensors"
# config["model"]["params"]["network_config"]["params"]["fine_tuning_method"] = None
model = instantiate_from_config(config.model)

/vol/paramonos2/projects/antoni/miniconda3/envs/svd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'model': {'target': 'sgm.models.diffusion.DiffusionEngine', 'params': {'scale_factor': 0.18215, 'disable_first_stage_autocast': True, 'denoiser_config': {'target': 'sgm.modules.diffusionmodules.denoiser.DiscreteDenoiser', 'params': {'num_idx': 1000, 'scaling_config': {'target': 'sgm.modules.diffusionmodules.denoiser_scaling.EpsScaling'}, 'discretization_config': {'target': 'sgm.modules.diffusionmodules.discretizer.LegacyDDPMDiscretization'}}}, 'network_config': {'target': 'sgm.modules.diffusionmodules.openaimodel.UNetModel', 'params': {'use_checkpoint': True, 'in_channels': 4, 'out_channels': 4, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_head_channels': 64, 'use_linear_in_transformer': False, 'transformer_depth': 1, 'context_dim': 768}}, 'conditioner_config': {'target': 'sgm.modules.GeneralConditioner', 'params': {'emb_models': [{'is_trainable': False, 'input_key': 'txt', 'target': 'sgm.modules.encoders.modules.Fr

In [1]:
import torch
from diffusers import StableDiffusionPipeline

pipeline_1_5 = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    # torch_dtype=torch.float16,
    cache_dir="/vol/paramonos2/projects/antoni/.cache",
).to("cuda")

prompt_embeds, negative_prompt_embeds = pipeline_1_5.encode_prompt(
    "Chybre",
    "cuda",
    1,
    False,
)
prompt_embeds

/vol/paramonos2/projects/antoni/miniconda3/envs/svd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 10.60it/s]


tensor([[[-0.3884,  0.0229, -0.0522,  ..., -0.4899, -0.3066,  0.0675],
         [-0.1189,  1.0663,  2.0914,  ...,  0.3393, -0.4146,  0.6376],
         [ 0.2818,  0.9080,  1.7518,  ...,  1.0331, -0.2975, -0.3620],
         ...,
         [ 0.9049, -0.0348,  0.5000,  ...,  2.3313, -0.7853, -0.6802],
         [ 0.8800, -0.0329,  0.5009,  ...,  2.3420, -0.8227, -0.6749],
         [ 0.9519,  0.0401,  0.5483,  ...,  2.2692, -0.7291, -0.6433]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [9]:
model = model.to("cuda")
emb = model.conditioner.embedders[0]("Chybre")
emb

tensor([[[-0.3883,  0.0229, -0.0520,  ..., -0.4898, -0.3066,  0.0674],
         [-0.1190,  1.0676,  2.0905,  ...,  0.3391, -0.4161,  0.6374],
         [ 0.2763,  0.9094,  1.7510,  ...,  1.0331, -0.2992, -0.3645],
         ...,
         [ 0.9047, -0.0336,  0.4978,  ...,  2.3328, -0.7846, -0.6800],
         [ 0.8800, -0.0305,  0.4989,  ...,  2.3419, -0.8218, -0.6741],
         [ 0.9512,  0.0413,  0.5462,  ...,  2.2694, -0.7275, -0.6435]]],
       device='cuda:0')

In [23]:
torch.abs(emb - prompt_embeds).mean()

tensor(0.0010, device='cuda:0', grad_fn=<MeanBackward0>)

In [29]:
pipeline_1_5

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.28.0.dev0",
  "_name_or_path": "runwayml/stable-diffusion-v1-5",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [27]:
model.model.diffusion_model

UNetModel(
  (time_embed): Sequential(
    (0): Linear(in_features=320, out_features=1280, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-2): 2 x TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=1280, out_features=320, bias=True)
        )
        (out_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0.0, inplace=False)
          (3): Conv2d(320, 320, kernel_size=(3